In [1]:
import gym
import random
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import backend as K
import numpy as np

In [2]:

def initiate_NN(learning_rate=1e-3):
    model = Sequential()
    model.add(Dense(64, input_dim=8, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(4, activation='linear'))
    model.compile(loss='mse', optimizer=Adam(lr=learning_rate))
    return model

def epsilon_action(epsilon):
    random_value = np.random.random()
    if random_value<=epsilon:
        action = env.action_space.sample()
    else:
        Y=online_model.predict(state.reshape(1,-1))
        action = np.argmax(Y[0])
    return action


In [3]:


gamma=0.99
epsilon_decay = 0.002
learning_rate =0.0002
max_e=0.8
min_e=0.0001
replay_batch_size=30
iterations=5000

In [4]:
env = gym.make('LunarLander-v2')
state=env.reset()
online_model = initiate_NN(learning_rate)
target_model =initiate_NN(learning_rate)
target_model.set_weights(online_model.get_weights())
memory=[]
episode_reward_rolling = []
episode_reward=[]

for episode in range(iterations):
    state=env.reset()
    current_episode_reward = 0
    done=False
    steps=0
    epsilon = min_e + (max_e - min_e)*np.exp(-epsilon_decay*episode)
    while not done:
        action = epsilon_action(epsilon)
        new_state, reward, done, info= env.step(action)
        steps+=1
        current_episode_reward +=reward
        memory.append([state,action,reward,new_state,done])
        state = new_state
        if len(memory)>=replay_batch_size:
            batch = random.sample(memory, replay_batch_size)
            batch = np.array(batch)
            non_terminal_samples = np.where(batch[:, 4] == False)
            y = np.copy(batch[:, 2])
            if len( non_terminal_samples[0]) > 0:
                #use online q values to select best action
                online_q = online_model.predict(np.vstack(batch[:, 3]))
                #use offline q values of best action as target 
                target_q = target_model.predict(np.vstack(batch[:, 3]))
                y[non_terminal_samples] += np.multiply(gamma, \
                            target_q[non_terminal_samples, \
                            np.argmax(online_q[non_terminal_samples, :][0], axis=1)][0])
            actions = np.array(batch[:, 1], dtype=int)
            y_target = online_model.predict(np.vstack(batch[:, 0]))
            y_target[range(replay_batch_size), actions] = y
            online_model.fit(np.vstack(batch[:, 0]), y_target, epochs=1, verbose=0)
        if done:
            target_model.set_weights(online_model.get_weights())

    episode_reward.append(current_episode_reward) 
    if len(memory)>=5000:
        memory = memory[-5000:]
    if len(episode_reward)>=100:
        episode_reward_rolling.append(np.average(episode_reward[-100:]))
    else:
        episode_reward_rolling.append(np.average(episode_reward))
    print('episode: ', episode, ' episode reward: ', '%.2f' % current_episode_reward, \
          ' rolling reward: ', '%.2f' % episode_reward_rolling[-1], ' Steps: ', steps, ' epsilon: ',\
          '%.2f' % epsilon)

episode:  0  episode reward:  -144.47  rolling reward:  -144.47  Steps:  101  epsilon:  0.80
episode:  1  episode reward:  -417.25  rolling reward:  -280.86  Steps:  87  epsilon:  0.80
episode:  2  episode reward:  -232.99  rolling reward:  -264.90  Steps:  155  epsilon:  0.80
episode:  3  episode reward:  -139.89  rolling reward:  -233.65  Steps:  98  epsilon:  0.80
episode:  4  episode reward:  -599.64  rolling reward:  -306.85  Steps:  101  epsilon:  0.79
episode:  5  episode reward:  -217.43  rolling reward:  -291.94  Steps:  165  epsilon:  0.79
episode:  6  episode reward:  -222.63  rolling reward:  -282.04  Steps:  94  epsilon:  0.79
episode:  7  episode reward:  -92.67  rolling reward:  -258.37  Steps:  76  epsilon:  0.79
episode:  8  episode reward:  -216.32  rolling reward:  -253.70  Steps:  120  epsilon:  0.79
episode:  9  episode reward:  -621.89  rolling reward:  -290.52  Steps:  115  epsilon:  0.79
episode:  10  episode reward:  -39.78  rolling reward:  -267.72  Steps:  11

episode:  88  episode reward:  -89.82  rolling reward:  -144.94  Steps:  120  epsilon:  0.67
episode:  89  episode reward:  -32.25  rolling reward:  -143.69  Steps:  128  epsilon:  0.67
episode:  90  episode reward:  -106.91  rolling reward:  -143.28  Steps:  133  epsilon:  0.67
episode:  91  episode reward:  -97.27  rolling reward:  -142.78  Steps:  155  epsilon:  0.67
episode:  92  episode reward:  28.76  rolling reward:  -140.94  Steps:  1000  epsilon:  0.67
episode:  93  episode reward:  -182.77  rolling reward:  -141.39  Steps:  134  epsilon:  0.66
episode:  94  episode reward:  -49.99  rolling reward:  -140.42  Steps:  146  epsilon:  0.66
episode:  95  episode reward:  -72.73  rolling reward:  -139.72  Steps:  75  epsilon:  0.66
episode:  96  episode reward:  22.43  rolling reward:  -138.05  Steps:  164  epsilon:  0.66
episode:  97  episode reward:  -22.93  rolling reward:  -136.87  Steps:  90  epsilon:  0.66
episode:  98  episode reward:  -63.12  rolling reward:  -136.13  Steps:

episode:  177  episode reward:  -23.34  rolling reward:  -57.25  Steps:  133  epsilon:  0.56
episode:  178  episode reward:  -183.09  rolling reward:  -58.15  Steps:  259  epsilon:  0.56
episode:  179  episode reward:  -193.75  rolling reward:  -59.36  Steps:  897  epsilon:  0.56
episode:  180  episode reward:  -21.75  rolling reward:  -59.25  Steps:  94  epsilon:  0.56
episode:  181  episode reward:  -62.56  rolling reward:  -59.01  Steps:  110  epsilon:  0.56
episode:  182  episode reward:  -59.14  rolling reward:  -59.20  Steps:  156  epsilon:  0.56
episode:  183  episode reward:  -65.44  rolling reward:  -58.27  Steps:  118  epsilon:  0.55
episode:  184  episode reward:  -74.42  rolling reward:  -58.17  Steps:  160  epsilon:  0.55
episode:  185  episode reward:  -80.79  rolling reward:  -58.50  Steps:  174  epsilon:  0.55
episode:  186  episode reward:  -104.10  rolling reward:  -58.64  Steps:  122  epsilon:  0.55
episode:  187  episode reward:  -12.16  rolling reward:  -57.51  Ste

episode:  266  episode reward:  -49.42  rolling reward:  -33.74  Steps:  255  epsilon:  0.47
episode:  267  episode reward:  -77.99  rolling reward:  -33.99  Steps:  120  epsilon:  0.47
episode:  268  episode reward:  -67.62  rolling reward:  -33.97  Steps:  108  epsilon:  0.47
episode:  269  episode reward:  29.16  rolling reward:  -33.45  Steps:  1000  epsilon:  0.47
episode:  270  episode reward:  -19.67  rolling reward:  -33.58  Steps:  183  epsilon:  0.47
episode:  271  episode reward:  -32.39  rolling reward:  -33.66  Steps:  1000  epsilon:  0.47
episode:  272  episode reward:  -80.04  rolling reward:  -34.56  Steps:  128  epsilon:  0.46
episode:  273  episode reward:  -30.51  rolling reward:  -34.82  Steps:  173  epsilon:  0.46
episode:  274  episode reward:  -89.63  rolling reward:  -35.08  Steps:  209  epsilon:  0.46
episode:  275  episode reward:  -78.67  rolling reward:  -35.61  Steps:  117  epsilon:  0.46
episode:  276  episode reward:  69.11  rolling reward:  -34.22  Steps

episode:  355  episode reward:  -96.88  rolling reward:  2.28  Steps:  239  epsilon:  0.39
episode:  356  episode reward:  -13.33  rolling reward:  2.85  Steps:  254  epsilon:  0.39
episode:  357  episode reward:  -159.03  rolling reward:  1.79  Steps:  1000  epsilon:  0.39
episode:  358  episode reward:  111.09  rolling reward:  3.35  Steps:  1000  epsilon:  0.39
episode:  359  episode reward:  -303.63  rolling reward:  0.64  Steps:  849  epsilon:  0.39
episode:  360  episode reward:  -87.00  rolling reward:  -0.36  Steps:  169  epsilon:  0.39
episode:  361  episode reward:  93.33  rolling reward:  1.00  Steps:  1000  epsilon:  0.39
episode:  362  episode reward:  -18.60  rolling reward:  0.67  Steps:  67  epsilon:  0.39
episode:  363  episode reward:  61.48  rolling reward:  1.12  Steps:  1000  epsilon:  0.39
episode:  364  episode reward:  66.91  rolling reward:  0.98  Steps:  1000  epsilon:  0.39
episode:  365  episode reward:  51.50  rolling reward:  2.03  Steps:  1000  epsilon:  

...

episode:  890  episode reward:  294.59  rolling reward:  172.63  Steps:  545  epsilon:  0.13
episode:  891  episode reward:  301.23  rolling reward:  179.86  Steps:  390  epsilon:  0.13
episode:  892  episode reward:  272.54  rolling reward:  179.75  Steps:  604  epsilon:  0.13
episode:  893  episode reward:  21.77  rolling reward:  179.87  Steps:  136  epsilon:  0.13
episode:  894  episode reward:  32.55  rolling reward:  177.74  Steps:  145  epsilon:  0.13
episode:  895  episode reward:  236.70  rolling reward:  180.13  Steps:  321  epsilon:  0.13
episode:  896  episode reward:  13.68  rolling reward:  177.74  Steps:  132  epsilon:  0.13
episode:  897  episode reward:  269.18  rolling reward:  178.08  Steps:  255  epsilon:  0.13
episode:  898  episode reward:  12.52  rolling reward:  181.12  Steps:  145  epsilon:  0.13
episode:  899  episode reward:  247.90  rolling reward:  181.11  Steps:  366  epsilon:  0.13
episode:  900  episode reward:  -42.41  rolling reward:  178.53  Steps:  2

episode:  979  episode reward:  276.78  rolling reward:  192.32  Steps:  211  epsilon:  0.11
episode:  980  episode reward:  152.31  rolling reward:  192.21  Steps:  1000  epsilon:  0.11
episode:  981  episode reward:  198.49  rolling reward:  191.85  Steps:  344  epsilon:  0.11
episode:  982  episode reward:  278.72  rolling reward:  192.26  Steps:  378  epsilon:  0.11
episode:  983  episode reward:  26.94  rolling reward:  191.50  Steps:  94  epsilon:  0.11
episode:  984  episode reward:  1.32  rolling reward:  188.56  Steps:  147  epsilon:  0.11
episode:  985  episode reward:  225.12  rolling reward:  188.17  Steps:  322  epsilon:  0.11
episode:  986  episode reward:  264.71  rolling reward:  190.48  Steps:  212  epsilon:  0.11
episode:  987  episode reward:  11.25  rolling reward:  188.16  Steps:  103  epsilon:  0.11
episode:  988  episode reward:  -66.69  rolling reward:  187.16  Steps:  118  epsilon:  0.11
episode:  989  episode reward:  179.50  rolling reward:  186.67  Steps:  5

episode:  1067  episode reward:  162.77  rolling reward:  200.58  Steps:  765  epsilon:  0.09
episode:  1068  episode reward:  172.10  rolling reward:  199.50  Steps:  493  epsilon:  0.09
episode:  1069  episode reward:  13.00  rolling reward:  197.36  Steps:  110  epsilon:  0.09
episode:  1070  episode reward:  -9.37  rolling reward:  194.65  Steps:  105  epsilon:  0.09
episode:  1071  episode reward:  -42.37  rolling reward:  191.36  Steps:  120  epsilon:  0.09
episode:  1072  episode reward:  2.36  rolling reward:  188.57  Steps:  140  epsilon:  0.09
episode:  1073  episode reward:  -5.21  rolling reward:  186.10  Steps:  120  epsilon:  0.09
episode:  1074  episode reward:  23.59  rolling reward:  185.98  Steps:  102  epsilon:  0.09
episode:  1075  episode reward:  23.57  rolling reward:  186.18  Steps:  124  epsilon:  0.09
episode:  1076  episode reward:  19.11  rolling reward:  183.83  Steps:  127  epsilon:  0.09
episode:  1077  episode reward:  90.95  rolling reward:  182.32  Ste

episode:  1155  episode reward:  217.74  rolling reward:  161.19  Steps:  245  epsilon:  0.08
episode:  1156  episode reward:  280.37  rolling reward:  163.87  Steps:  317  epsilon:  0.08
episode:  1157  episode reward:  253.80  rolling reward:  163.79  Steps:  222  epsilon:  0.08
episode:  1158  episode reward:  222.23  rolling reward:  165.91  Steps:  362  epsilon:  0.08
episode:  1159  episode reward:  -15.00  rolling reward:  165.46  Steps:  171  epsilon:  0.08
episode:  1160  episode reward:  1.04  rolling reward:  165.74  Steps:  162  epsilon:  0.08
episode:  1161  episode reward:  28.06  rolling reward:  166.14  Steps:  162  epsilon:  0.08
episode:  1162  episode reward:  -9.09  rolling reward:  165.57  Steps:  151  epsilon:  0.08
episode:  1163  episode reward:  237.39  rolling reward:  168.14  Steps:  319  epsilon:  0.08
episode:  1164  episode reward:  -6.48  rolling reward:  166.06  Steps:  155  epsilon:  0.08
episode:  1165  episode reward:  271.02  rolling reward:  166.52 

KeyboardInterrupt: 

In [9]:
np.array(episode_reward_rolling[-20:])>200


array([False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [7]:
online_model.save_weights('weights_DQN_'+str(max_e)+'-'+str(min_e)+'-'+str(epsilon_decay)+'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.h5')

import pickle

with open('weights_DQN_'+str(max_e)+'-'+str(min_e)+'-'+str(epsilon_decay)+'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.json', 'wb') as fp:
    pickle.dump({'rewards':episode_reward,'rolling_reward':episode_reward_rolling}, fp)


In [167]:
rward = pickle.load( open( 'weights_DQN_'+str(max_e)+'-'+str(min_e)+'-'+str(epsilon_decay)\
                         +'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.json', "rb" ) )

In [168]:
rward

{'rewards': [-131.3223570990876, -248.23258991485326, -133.48636655895385],
 'rolling_reward': [-131.3223570990876,
  -189.77747350697044,
  -171.01377119096492]}

In [163]:
episode_reward

[]

In [152]:
Y_target.shape

(5, 4)

In [142]:
batch[terminal_samples].shape

(0, 5)

In [125]:



# i:index for sample [state,action,reward,new_state,done]
for i,sample in enumerate(batch):
    terminal_sample = sample[4]
    new_state_sample = sample[3]
    state_sample = sample[0]
    X[i]=state_sample
    action_sample = sample[1]
    Y_target[i] = online_model.predict(state_sample.reshape(1,-1))[0]
    if terminal:
        #terminal then targetY is reward
        Y_target[i][action_sample] = sample[2]
    else:
        #use online weights to select argmax action under new_state

        online_qvalues_batch =online_model.predict(new_state_sample.reshape(1,-1)) 
        max_action = np.argmax(online_qvalues_batch)
        #use target weights to calculate Qvalue
        #non-terminal then targetY is reward +gamma*new Qvalue
        Y_target[i][action_sample] = sample[2] + gamma * target_model.predict(new_state_sample.reshape(1,-1)).reshape(4,)[max_action]

online_model.fit(np.array(X), np.array(Y_target), verbose=0)


100

In [ ]:
Notice that the selection of the action, in the argmax, is


In [91]:
online_qvalues_batch

[array([[ 0.18740644, -0.2721934 ,  0.2233217 , -0.45678362]],
       dtype=float32),
 array([[ 0.19613695, -0.27545747,  0.20470577, -0.46681872]],
       dtype=float32),
 array([[ 0.18746991, -0.25133634,  0.19952038, -0.4359238 ]],
       dtype=float32),
 array([[ 0.19334945, -0.2700162 ,  0.19816679, -0.45707738]],
       dtype=float32),
 array([[ 0.19308917, -0.27386588,  0.21208042, -0.46272066]],
       dtype=float32)]

In [113]:
np.array(X)

array([[-0.00213823,  1.4003743 , -0.21659775, -0.4687014 ,  0.0024845 ,
         0.04906261,  0.        ,  0.        ],
       [-0.00846777,  1.3651963 , -0.20548455, -0.5481141 ,  0.00758275,
         0.00502167,  0.        ,  0.        ],
       [-0.01887293,  1.3004851 , -0.22156799, -0.61512053,  0.00625335,
         0.04479944,  0.        ,  0.        ],
       [-0.01250668,  1.3402183 , -0.20310256, -0.5355056 ,  0.00488083,
        -0.02924636,  0.        ,  0.        ],
       [-0.00641537,  1.3775289 , -0.21631236, -0.5210343 ,  0.00733169,
         0.04846335,  0.        ,  0.        ]], dtype=float32)

In [112]:
Y_target

[array([-1.2552428 , -0.2741652 ,  0.2263453 , -0.45686692], dtype=float32),
 array([ 0.19308917, -0.27386588,  0.21208042, -0.6722371 ], dtype=float32),
 array([ 0.1933487, -0.2546284,  5.1014786, -0.445849 ], dtype=float32),
 array([-0.91460276, -0.27277392,  0.20281965, -0.4571485 ], dtype=float32),
 array([ 0.18904   , -0.27001685,  0.21992686, -0.7534547 ], dtype=float32)]

In [ ]:

class DDQL:
    def __init__(self, nS, nA):
        self.nS = nS
        self.nA = nA
        self.epsilon = 1
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.9993
        self.gamma = 0.99
        self.learning_rate = 0.0001
        self.epochs = 1
        self.verbose = 0
        self.minibatch_size = 30
        self.memory = deque(maxlen=5000)
        self.model = self.create_model()
        self.target_model = self.create_model()

    def create_model(self):
        model = Sequential()

        # Add 2 hidden layers with 64 nodes each
        model.add(Dense(64, input_dim=self.nS, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(self.nA, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def add_memory(self, s, a, r, s_prime, done):
        self.memory.append((s, a, r, s_prime, done))

    def target_model_update(self):
        self.target_model.set_weights(self.model.get_weights())

    def selectAction(self, s):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.nA)
        q = self.model.predict(s)
        return np.argmax(q[0])

    def replay(self):
        # Vectorized method for experience replay
        minibatch = random.sample(self.memory, self.minibatch_size)
        minibatch = np.array(minibatch)
        not_done_indices = np.where(minibatch[:, 4] == False)
        y = np.copy(minibatch[:, 2])

        # If minibatch contains any non-terminal states, use separate update rule for those states
        if len(not_done_indices[0]) > 0:
            predict_sprime = self.model.predict(np.vstack(minibatch[:, 3]))
            predict_sprime_target = self.target_model.predict(np.vstack(minibatch[:, 3]))
            
            # Non-terminal update rule
            y[not_done_indices] += np.multiply(self.gamma, \
                        predict_sprime_target[not_done_indices, \
                        np.argmax(predict_sprime[not_done_indices, :][0], axis=1)][0])

        actions = np.array(minibatch[:, 1], dtype=int)
        y_target = self.model.predict(np.vstack(minibatch[:, 0]))
        y_target[range(self.minibatch_size), actions] = y
        self.model.fit(np.vstack(minibatch[:, 0]), y_target, epochs=self.epochs, verbose=self.verbose)

    def replayIterative(self):
        # Iterative method - this performs the same function as replay() but is not vectorized 
        s_list = []
        y_state_list = []
        minibatch = random.sample(self.memory, self.minibatch_size)
        for s, a, r, s_prime, done in minibatch:
            s_list.append(s)
            y_action = r
            if not done:
                y_action = r + self.gamma * np.amax(self.model.predict(s_prime)[0])

            print(y_action)
            
            y_state = self.model.predict(s)
            y_state[0][a] = y_action
            y_state_list.append(y_state)
        self.model.fit(np.squeeze(s_list), np.squeeze(y_state_list), batch_size=self.minibatch_size, epochs=1, verbose=0)

if __name__ == "__main__":
    np.set_printoptions(precision=2)
    env = gym.make('LunarLander-v2')
    env = wrappers.Monitor(env, tdir, force=True, video_callable=False)

    nS = env.observation_space.shape[0]
    nA = env.action_space.n

    agent = DDQL(nS, nA)

    # Set to true to use saved model
    episodes = 10000

    # Cumulative reward
    reward_avg = deque(maxlen=100)

    for e in range(episodes):
        episode_reward = 0
        s = env.reset()
        s = np.reshape(s, [1, nS])

        for time in range(1000):

            # Query next action from learner and perform action
            a = agent.selectAction(s)
            s_prime, r, done, info = env.step(a)

            # Add cumulative reward
            episode_reward += r

            # Reshape new state
            s_prime = np.reshape(s_prime, [1, nS])

            # Add experience to memory
            agent.add_memory(s, a, r, s_prime, done)

            # Set current state to new state
            s = s_prime

            #Perform experience replay if memory length is greater than minibatch length
   
            if len(agent.memory) > agent.minibatch_size:
                agent.replay()

            # If episode is done, exit loop
            if done:
                agent.target_model_update()
                break


        # epsilon decay
        if agent.epsilon > agent.epsilon_min:
            agent.epsilon *= agent.epsilon_decay

        # Running average of past 100 episodes
        reward_avg.append(episode_reward)
        print('episode: ', e, ' score: ', '%.2f' % episode_reward, ' avg_score: ', '%.2f' % np.average(reward_avg), ' frames: ', time, ' epsilon: ', '%.2f' % agent.epsilon)
        
        # with open('trained_agent.txt', 'a') as f:
        #     f.write(str(np.average(reward_avg)) + '\n')

    env.close()
#     gym.upload(tdir, api_key='sk_EJo79Jo0RsSEA4EF6Cp5mg')


#agent.model.save_weights('weights.h5')
#agent.model.load_weights('weights.h5')

episode:  0  score:  -101.20  avg_score:  -101.20  frames:  104  epsilon:  1.00
episode:  1  score:  -147.99  avg_score:  -124.59  frames:  74  epsilon:  1.00
episode:  2  score:  -95.67  avg_score:  -114.95  frames:  74  epsilon:  1.00
episode:  3  score:  -218.30  avg_score:  -140.79  frames:  94  epsilon:  1.00
episode:  4  score:  -231.98  avg_score:  -159.03  frames:  86  epsilon:  1.00
episode:  5  score:  -161.32  avg_score:  -159.41  frames:  68  epsilon:  1.00
episode:  6  score:  -96.64  avg_score:  -150.44  frames:  66  epsilon:  1.00
episode:  7  score:  -141.43  avg_score:  -149.32  frames:  97  epsilon:  0.99
episode:  8  score:  -480.81  avg_score:  -186.15  frames:  102  epsilon:  0.99
episode:  9  score:  -371.18  avg_score:  -204.65  frames:  92  epsilon:  0.99
episode:  10  score:  -161.56  avg_score:  -200.73  frames:  95  epsilon:  0.99
episode:  11  score:  -80.20  avg_score:  -190.69  frames:  61  epsilon:  0.99
episode:  12  score:  -332.47  avg_score:  -201.60 

episode:  103  score:  -168.96  avg_score:  -179.65  frames:  118  epsilon:  0.93
episode:  104  score:  -64.43  avg_score:  -177.97  frames:  91  epsilon:  0.93
episode:  105  score:  -200.70  avg_score:  -178.36  frames:  72  epsilon:  0.93
episode:  106  score:  32.45  avg_score:  -177.07  frames:  121  epsilon:  0.93
episode:  107  score:  -260.25  avg_score:  -178.26  frames:  117  epsilon:  0.93
episode:  108  score:  -110.97  avg_score:  -174.56  frames:  80  epsilon:  0.93
episode:  109  score:  -253.15  avg_score:  -173.38  frames:  94  epsilon:  0.93
episode:  110  score:  -125.12  avg_score:  -173.02  frames:  88  epsilon:  0.93
episode:  111  score:  -189.74  avg_score:  -174.11  frames:  72  epsilon:  0.92
episode:  112  score:  -140.07  avg_score:  -172.19  frames:  94  epsilon:  0.92
episode:  113  score:  -304.20  avg_score:  -173.17  frames:  71  epsilon:  0.92
episode:  114  score:  -128.94  avg_score:  -173.02  frames:  72  epsilon:  0.92
episode:  115  score:  -294.

episode:  204  score:  -298.51  avg_score:  -150.87  frames:  97  epsilon:  0.87
episode:  205  score:  -77.35  avg_score:  -149.64  frames:  146  epsilon:  0.87
episode:  206  score:  -124.22  avg_score:  -151.21  frames:  68  epsilon:  0.87
episode:  207  score:  -40.81  avg_score:  -149.01  frames:  144  epsilon:  0.86
episode:  208  score:  -83.08  avg_score:  -148.73  frames:  67  epsilon:  0.86
episode:  209  score:  -268.33  avg_score:  -148.89  frames:  92  epsilon:  0.86
episode:  210  score:  -124.07  avg_score:  -148.88  frames:  105  epsilon:  0.86
episode:  211  score:  -123.48  avg_score:  -148.21  frames:  122  epsilon:  0.86
episode:  212  score:  -143.46  avg_score:  -148.25  frames:  80  epsilon:  0.86
episode:  213  score:  -86.07  avg_score:  -146.07  frames:  96  epsilon:  0.86
episode:  214  score:  -86.35  avg_score:  -145.64  frames:  94  epsilon:  0.86
episode:  215  score:  -55.86  avg_score:  -143.25  frames:  93  epsilon:  0.86
episode:  216  score:  -58.61 